# 라이브러리 추가

In [1]:
import pandas as pd
import os
import sys
import json

# 파일 읽어오기   
* 파일 경로를 가져온다.
    * 만약 google drive를 통해 파일을 가져온다면 아래와 같이 파일 경로를 찾아 오자.
    * `!find /content/drive/MyDrive/ -name read/*`

In [8]:
file_path = input("file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/read)>>")

#read 디렉터리 내의 파일 리스트를 가지고 옵니다.
read_file_list = os.listdir(file_path)  
read_dict = {}  # key : 파일명, value : dict -> 
                #                           key : user id
                #                           value : 읽은 글 id

for file_name in read_file_list:
    tmp_path = os.path.join(file_path, file_name)
    with open(tmp_path, 'r') as f:
        read_dict[file_name] = {}
        for line in f:
            line_list = line.split()
            read_dict[file_name][line_list[0]] = []
            for i in range(1, len(line_list)):
                read_dict[file_name][line_list[0]].append(line_list[i])
        break#test용으로 하나의 파일만 불러와서 확인해봄

print(read_dict)

file path plz :)(ex) /content/drive/MyDrive/kakao arena/data/read)>>/content/drive/My Drive/한양대학교/브런치 사용자를 위한 글 추천 대회/data/read
{'2019012317_2019012318': {'#c000e5348c62b53efc6cbf930f4b9d7d': ['@uppity_4', '@moneyjagga_34'], '#60f7eebb4c0241f05f5be20cc94008d3': ['@rothem_661', '@rothem_661', '@brunch_148', '@rothem_660'], '#f3aeac4e07dadab788b95a3833583fed': ['@medtech_96', '@medtech_96'], '#af30673cade0b0be2ee78811ab3fa7bc': ['@reinia_41'], '#b165471f8b800280104745ac4a4dd179': ['@wikitree_1648', '@hosueng_82', '#46299aafa81870e19d00764047631c3f_29', '#46299aafa81870e19d00764047631c3f_28', '@sejindo_7', '@roysday_258', '@jade_203', '@anti-essay_21', '@jordan777_1501', '@socialmkt_72'], '#38caf952d8243f210b74abd9e98309fd': ['@taekangk_58', '@donko_152', '@taekangk_58', '@mustit_105', '@pyk8627_41', '@elinacho_13', '@hakuna_27', '@pyk8627_41', '@suhakang_35', '@suhakang_35', '@studiocroissant_41', '@shinnarious_86', '@conbus_43', '@conbus_43', '@psychiatricnews_18', '@psychiatricnews_18'

* read 디렉토리: 본 글 정보

  * read.tar은 2018년 10월 1일부터 2019년 3월 1일까지 일부 브런치 독자들이 본 글의 정보가 총 3,625개의 파일로 구성되어 있습니다.
  * 파일의 이름은 시작일_종료일 형태입니다. 예를 들어 2018110708_2018110709 파일은 2018년 11월 7일 오전 8시부터 2018년 11월 7일 오전 9시 전까지 본 글입니다.
  * 파일은 여러 줄로 이뤄져 있으며 하나의 줄은 브런치의 독자가 파일의 시간 동안 본 글을 시간 순으로 기록한 것입니다. 한 줄의 정보는 공백으로 구분되어 있으며 첫번 째가 독자의 암호화된 식별자이고 그 뒤로는 해당 독자가 본 글의 정보입니다.
  * 예를 들어 read/2019022823_2019030100 파일에 기록된 아래 정보는 #8a706ac921a11004bab941d22323efab 라는 독자가 2019년 2월 28일 23시에서 2019년 3월 1일 0시 사이에 @bakchacruz_34 @wo-motivator_133 @wo-motivator_133 를 순서대로 보았다는 뜻입니다. @wo-motivator_133 글이 두 번 나타난 것은 이 글을 보기 위해 두 번 방문했다는 뜻입니다.
    * #8a706ac921a11004bab941d22323efab @bakchacruz_34 @wo-motivator_133 @wo-motivator_133
  * 글을 보았다는 의미는 특정 글에 모바일, PC, 앱을 통해 접근했다는 뜻입니다. 머문 시간에 대한 정보가 제공되지 않기 때문에 실제로 글을 읽지 않고 이탈하는 등의 가능성도 있습니다.